# Hand recognition - Dev Sliders

In [1]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
def extract_hand(img, Hlower=0, Slower=27, Vlower=25, Hupper=9, Supper=214, Vupper=162, dil1_iter=2, 
                     erode1_iter=1, erode2_iter=1, dil2_iter=5, dil3_iter=4, dil4_iter=2, resize = True):
    if resize:
        # Optional - resize the image by half
        img = cv2.resize(img, (0,0), fx=0.5, fy=0.5) 

    #Convert to HSV color space
    hsv_frame = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    #Create a binary image with where white will be skin colors and rest is black
    mask = cv2.inRange(hsv_frame,np.array([Hlower,Slower,Vlower]),np.array([Hupper,Supper,Vupper]))

    #Kernel matrices for morphological transformation    
    kernel_square = np.ones((11,11),np.uint8)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))

    #Perform morphological transformations to filter out the background noise
    f = cv2.dilate(mask,kernel_ellipse,iterations = dil1_iter)
    f = cv2.erode(f,kernel_square,iterations = erode1_iter)    
    f = cv2.dilate(f,kernel_ellipse,iterations = dil2_iter)  
    f = cv2.erode(f,kernel_square,iterations = erode1_iter)    
    f = cv2.medianBlur(f,15)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(8,8))
    f = cv2.dilate(f,kernel_ellipse,iterations = dil3_iter)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    f = cv2.erode(f,kernel_square,iterations = erode2_iter)   
    f = cv2.dilate(f,kernel_ellipse,iterations = dil4_iter)
    f = cv2.medianBlur(f,15)
    threshold = cv2.threshold(f,127,255,0)[1]
#     cv2.imshow('BGclean',ret)

    #Find contours of the filtered frame
    _, contours, hierarchy = cv2.findContours(threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)   

    #Find Max contour area (Assume that hand is in the frame)
    max_area=100
    ci=0	
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        if(area>max_area):
            max_area=area
            ci=i  

    try:
        cnts = contours[ci]
    except IndexError:
        print("No contour found")
        return img, img
    #Largest area contour  
    cnts = contours[ci]

    #Print bounding rectangle
    # x,y is top left corner
    x,y,w,h = cv2.boundingRect(cnts)

    # Green Rectangle
#     frame = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

    # The idea here is to see if the crop is wider or taller. based on this, try as best as possible to crop
    # a square. This is only a problem when we are near the edge of the image.

    if h > w:
        x_center = x + w/2.0
        x_start = int(x_center - h*0.5)
        x_end = int(x_center + h*0.5) 
        y_start = y
        y_end = y + h
        # light blue rectangle
        img = cv2.rectangle(img, (x_start, y_start), (x_end, y_end), (255, 255, 0), 2)
    else:
        y_center = y + h/2.0
        x_start = x
        x_end = x + w
        y_start = int(y_center - w*0.5)
        y_end = int(y_center + w*0.5)
        # yellow rectangle
        img = cv2.rectangle(img, (x_start, y_start), (x_end, y_end), (0, 255, 255), 2)

#     print('x',x,'y',y, 'h',h,'w',w)

    # Cropping notes:
    # When cropping the image, x & y start at top left.
    # y is from top to buttom of the image
    # x is from lef to right
    # These only apply if we put Y first when indexing: crop = img[ystart:ystop, xstart:xstop]

    crop = img[y_start : y_end, x_start : x_end]
                        
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     cv2.putText(frame,'TPF:' + str(time.time()-start_time),(10,50),font,1,(255,255,255),2)     

    try:
        crop = cv2.resize(crop, (200, 200))
    except:
        crop = img[0 : 10, 0 : 10]
        crop = cv2.resize(crop, (200, 200))
    return img, crop
    
def create_trackbar(name, default_value, l_limit = 0, u_limit = 255):
    cv2.createTrackbar(name,'TrackBars',l_limit, u_limit, nothing)
    cv2.setTrackbarPos(name, 'TrackBars', default_value)
    
def nothing(x):
    pass

In [ ]:
# Creating a window for HSV track bars
cv2.namedWindow('TrackBars', cv2.WINDOW_FREERATIO)

create_trackbar('Hlower', 0)
create_trackbar('Slower', 27)
create_trackbar('Vlower', 25)
create_trackbar('Hupper', 9)
create_trackbar('Supper', 214)
create_trackbar('Vupper', 162)

create_trackbar('dilate1_iterations', 2, u_limit = 10)
create_trackbar('dilate2_iterations', 5, u_limit = 10)
create_trackbar('dilate3_iterations', 4, u_limit = 10)
create_trackbar('dilate4_iterations', 2, u_limit = 10)
create_trackbar('erode1_iterations', 1, u_limit = 10)
create_trackbar('erode2_iterations', 1, u_limit = 10)


while True:
    start_time = time.time()

    Hlower = cv2.getTrackbarPos('Hlower','TrackBars')
    Slower = cv2.getTrackbarPos('Slower','TrackBars')
    Vlower = cv2.getTrackbarPos('Vlower','TrackBars')
    Hupper = cv2.getTrackbarPos('Hupper','TrackBars')
    Supper = cv2.getTrackbarPos('Supper','TrackBars')
    Vupper = cv2.getTrackbarPos('Vupper','TrackBars')
    dil3_iter = cv2.getTrackbarPos('dilate3_iterations','TrackBars')
    dil4_iter = cv2.getTrackbarPos('dilate4_iterations','TrackBars')
    dil1_iter = cv2.getTrackbarPos('dilate1_iterations','TrackBars')
    erode1_iter = cv2.getTrackbarPos('erode1_iterations','TrackBars')
    erode2_iter = cv2.getTrackbarPos('erode2_iterations','TrackBars')
    dil2_iter = cv2.getTrackbarPos('dilate2_iterations','TrackBars')

    img1 = cv2.imread('/Users/idofarhi/Documents/ITC/Projects/ASL/crop_test_images/1.jpg')
    frame, crop = extract_hand(img1, Hlower, Slower, Vlower, Hupper, Supper, Vupper, dil1_iter, erode1_iter, erode2_iter, dil2_iter, dil3_iter, dil4_iter)
    cv2.imshow('original image 1', frame)
#     cv2.imshow('crop 1',crop)
    
    img2 = cv2.imread('/Users/idofarhi/Documents/ITC/Projects/ASL/crop_test_images/2.jpg')
    frame, crop = extract_hand(img2, Hlower, Slower, Vlower, Hupper, Supper, Vupper, dil1_iter, erode1_iter, erode2_iter, dil2_iter, dil3_iter, dil4_iter, False)
    cv2.imshow('original image 2', frame)
#     cv2.imshow('crop 2',crop)
    
    img3 = cv2.imread('/Users/idofarhi/Documents/ITC/Projects/ASL/crop_test_images/3.jpg')
    frame, crop = extract_hand(img3, Hlower, Slower, Vlower, Hupper, Supper, Vupper, dil1_iter, erode1_iter, erode2_iter, dil2_iter, dil3_iter, dil4_iter)
    cv2.imshow('original image 3', frame)
#     cv2.imshow('crop 3',crop)
    
    img4 = cv2.imread('/Users/idofarhi/Documents/ITC/Projects/ASL/crop_test_images/4.jpg')
    frame, crop = extract_hand(img4, Hlower, Slower, Vlower, Hupper, Supper, 
                               Vupper, dil1_iter, erode1_iter, erode2_iter, dil2_iter, dil3_iter, dil4_iter, False)
    cv2.imshow('original image 4', frame)
#     cv2.imshow('crop 4',crop)
    
    img5 = cv2.imread('/Users/idofarhi/Documents/ITC/Projects/ASL/crop_test_images/5.jpg')
    frame, crop = extract_hand(img5, Hlower, Slower, Vlower, Hupper, Supper, 
                               Vupper, dil1_iter, erode1_iter, erode2_iter, dil2_iter, dil3_iter, dil4_iter, False)
    cv2.imshow('original image 5', frame)
#     cv2.imshow('crop 5',crop)
    
    img6 = cv2.imread('/Users/idofarhi/Documents/ITC/Projects/ASL/crop_test_images/6.jpg')
    frame, crop = extract_hand(img6, Hlower, Slower, Vlower, Hupper, Supper, Vupper, dil1_iter, erode1_iter, erode2_iter, dil2_iter, dil3_iter, dil4_iter, False)
    cv2.imshow('original image 6', frame)
#     cv2.imshow('crop 6',crop)
    
#     img7 = cv2.imread('/Users/idofarhi/Documents/ITC/Projects/ASL/crop_test_images/7.jpg')
#     frame, crop = extract_hand(img7, Hlower, Slower, Vlower, Hupper, Supper, Vupper, dil1_iter, erode1_iter, erode2_iter, dil2_iter, dil3_iter, dil4_iter)
#     cv2.imshow('original image 7', frame)
#     cv2.imshow('crop 7',crop)
    
    img8 = cv2.imread('/Users/idofarhi/Documents/ITC/Projects/ASL/crop_test_images/8.jpg')
    frame, crop = extract_hand(img8, Hlower, Slower, Vlower, Hupper, Supper, Vupper, dil1_iter, erode1_iter, erode2_iter, dil2_iter, dil3_iter, dil4_iter)
    cv2.imshow('original image 8', frame)
#     cv2.imshow('crop 8',crop)
    
    # Press q to quit, p to print the current HSV values
    if cv2.waitKey(100) & 0xFF == ord('p'):
        print(Hlower,Slower,Vlower, Hupper,Supper,Vupper, dil3_iter, dil4_iter)

    #close the output video by pressing 'q'
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# cv2.destroyAllWindows()
# cv2.waitKey(1)